### D_report

В данном блокноте содержится реально используемый алгоритм, состоящий из следующих этапов:
- Получение данных нового периода из различных источников
- Преобразования в стандартный формат
- Обьединение с историческими данными
- Расчёт стоимостей
- Расчёт показателей YTD, Grt
- Проверка корректности результатов каждого этапа
- Преобразовани в утверждённый формат отчётности

###### Содержание 
- 1) Грузим main (основной файл)
- 2) Грузим Price, Prodmap
- 3) Записываем функции:
    - files_from_folder_to_df,
    - delete_totals,
    - find_not_mapped,
    - pivoted
    - YTD_LY
    - YTD_2021
    - Grt_YTD
- 4) Грузим и стандартизируем исходные отчёты
- 5) Сцепляем в единый "monthly_df"
- 6) Предпроверка
- 7) Присоединяем montly_df к main
- 8) Задаём Cur_mon, Cur_mon_LY 
- 9) Постпроверка
- 10) Расчёт Sales YTD & YTD_LY
- 11) Оставляем Cur_mon и YTD
- 12) Рассчитываем Тотал по дистрам
- 13) Присоединяем Тотал к _ selected_packs
- 14) Расчёт Руб
- 15) Добавляем Тотал по продуктам в Уп и Руб
- 16) Добавляем приросты
- 17) Выстраиваем в нужный вид
- 18) Делим D на: month и YTD

In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_rows', 150)
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
#Устанавливаем текущий период
cur_per = '2021_03'

# 1) Грузим main
основной файл с продажами  (прошлого года + этого по предыдущий месяц)

In [4]:
os.chdir(r"C:\Data\Илья\DataScience\D_report\00_D")
main = pd.read_excel('Data_enc.xlsx')

In [5]:
main.tail()

,Distributor,Product_id_SS,Product_name_SS,S1904P,S1905P,S1906P,S1907P,S1908P,S1909P,S1910P,...,S2005P,S2006P,S2007P,S2008P,S2009P,S2010P,S2011P,S2012P,S2101P,S2102P
941,D9,300025095,Redinesp 5,691,656,305,603,516,742,197,...,337,503,286,958,739,529,966,193,320,752
942,D9,300009262,"Senade tab 13,5mg №500",164,476,890,346,179,775,948,...,32,383,535,288,266,11,899,16,397,855
943,D9,300000727,Sparflo tab f/c 200mg №6,384,410,499,910,849,134,29,...,526,60,401,225,323,622,962,864,750,897
944,D9,300029824,Texared injection 20 mg №1,538,177,328,792,195,616,966,...,662,516,209,817,372,665,228,790,774,193
945,D9,300030128,Texared tablets 20 mg №10,972,932,656,366,754,276,116,...,427,456,518,406,514,789,627,626,200,715


# 2) Грузим Price, Prodmap

###### Prices

In [6]:
os.chdir(r"C:\Data\Илья\DataScience\D_report\00_D")
prices = pd.read_excel('Prices_enc.xlsx')

In [7]:
prices.head()

,Product_id_SS,Product_name_SS,Price_fix_RUB
0,300027136,Allerway 5 mg tab №10,199
1,300027158,Allerway 5 mg tab №30,810
2,300031318,Allerway Express (1x10s)Tab Russia,408
3,300001611,Bion 3 tab №10,368
4,300001761,Bion 3 tab №30,443


### Prodmap

In [8]:
os.chdir(r"C:\Data\Илья\DataScience\D_report\00_D")
prodmap = pd.read_excel('Prodmap_enc.xlsx')
prodmap.Product_name_origin = prodmap.Product_name_origin.str.lower()

# 3) Записываем Функции

In [10]:
def files_from_folder_to_df(path):
    os.chdir(path )
    report_list = os.listdir() 
    list_of_dfs = []
    for file in report_list:
        df = pd.read_excel(file)
        list_of_dfs.append(df)
    return pd.concat(list_of_dfs, ignore_index=True)

In [11]:
def delete_totals(col_name: str):
    return big_df[big_df[col_name].notnull()]

In [12]:
def find_not_mapped(col_name: str):
    not_mapped = big_df_ss[big_df_ss.Product_id_SS.isnull()][col_name].unique()
    if len(not_mapped)==0:
        print('ОК. Все продукты смепплены!')
        return
    nm = (pd.Series(not_mapped, name = 'Product')).sort_values()
    print(nm)
    raise SystemExit("Возьми Not_mapped и дополни Prodmap!")

In [13]:
def pivoted(qty_col: str):
    pi_df =  big_df_ss.pivot_table(index = ['Distributor','Month','Product_id_SS','Product_name_SS'],
                                   values = qty_col, aggfunc = 'sum', fill_value=0)
    return pi_df.reset_index()

In [14]:
def YTD_1920(mon):
    x = main.S1904P
    if mon == 'Apr':
        return x
    x += main.S1905P
    if mon == 'May':
        return x
    x += main.S1906P
    if mon == 'Jun':
        return x
    x += main.S1907P
    if mon == 'Jul':
        return x
    x += main.S1908P
    if mon == 'Aug':
        return x  
    x += main.S1909P
    if mon == 'Sep':
        return x
    x += main.S1910P
    if mon == 'Oct':
        return x  
    x += main.S1911P
    if mon == 'Nov':
        return x
    x += main.S1912P
    if mon == 'Dec':
        return x
    x += main.S2001P
    if mon == 'Jan':
        return x
    x += main.S2002P
    if mon == 'Feb':
        return x
    x += main.S2003P
    if mon == 'Mar':
        return x

In [15]:
def YTD_2021(mon):
    x = main.S2004P
    if mon == 'Apr':
        return x
    x += main.S2005P
    if mon == 'May':
        return x
    x += main.S2006P
    if mon == 'Jun':
        return x
    x += main.S2007P
    if mon == 'Jul':
        return x
    x += main.S2008P
    if mon == 'Aug':
        return x  
    x += main.S2009P
    if mon == 'Sep':
        return x
    x += main.S2010P
    if mon == 'Oct':
        return x  
    x += main.S2011P
    if mon == 'Nov':
        return x
    x += main.S2012P
    if mon == 'Dec':
        return x
    x += main.S2101P
    if mon == 'Jan':
        return x
    x += main.S2102P
    if mon == 'Feb':
        return x
    x += main.S2103P
    if mon == 'Mar':
        return x

In [16]:
def Grt_YTD(row):
    if selected_all.YTD_LY[row] == 0:
        return 0
    return selected_all.YTD[row] / selected_all.YTD_LY[row] - 1

In [17]:
def Grt_month(row):
    if selected_all.Cur_mon_LY[row] == 0:
        return 0
    return (selected_all.Cur_mon[row] / selected_all.Cur_mon_LY[row] - 1)

# 4) Грузим и трансформируем в стандарт исходные отчёты дистров 

## D1

In [18]:
path = r"C:\Data\Илья\DataScience\D_report\01_D"
col_name ='NM_MED'
qty_col = "Qty"

In [19]:
big_df = files_from_folder_to_df(path)
big_df['Month'] = cur_per
big_df['Distributor'] = 'D1'

In [20]:
big_df_ss = delete_totals(col_name)
big_df_ss[col_name] = big_df_ss[col_name].str.lower() 

In [21]:
big_df_ss = pd.merge(big_df_ss, prodmap, how ='left', left_on = col_name, right_on = 'Product_name_origin')
find_not_mapped(col_name)

ОК. Все продукты смепплены!


In [22]:
# Только для данного дистра
big_df_ss['Qty'] = big_df_ss.SALES - big_df_ss.RETURN

In [23]:
pi_df = pivoted(qty_col)
D1 = pi_df.copy()

In [24]:
D1 = D1.rename(columns={qty_col: 'Packs'})
D1.sort_values('Product_name_SS')[:5]

,Distributor,Month,Product_id_SS,Product_name_SS,Packs
58,D1,2021_03,300027136,Allerway 5 mg tab №10,150
60,D1,2021_03,300027158,Allerway 5 mg tab №30,158
79,D1,2021_03,300031318,Allerway Express (1x10s)Tab Russia,66
15,D1,2021_03,300001611,Bion 3 tab №10,84
17,D1,2021_03,300001761,Bion 3 tab №30,100


## D2

In [25]:
# создаём df из всех файлов папки
path = r"C:\Data\Илья\DataScience\D_report\02_D"
big_df = files_from_folder_to_df(path)

In [26]:
big_df['Month'] = cur_per
big_df['Distributor'] = 'D2'

In [27]:
# удаляем строки итогов
col_name ='СКЮ'
big_df_ss = delete_totals(col_name)
big_df_ss[col_name] = big_df_ss[col_name].str.lower()

In [28]:
# прицепляем Prodmap
big_df_ss = pd.merge(big_df_ss, prodmap, how ='left', left_on = col_name, right_on = 'Product_name_origin')

In [29]:
# проверка несмеппленых продуктов: либо создаём файл, либо получаем сообщение что всё Ок. 
find_not_mapped(col_name)

ОК. Все продукты смепплены!


In [30]:
#Добавляем несмеппленные продукты в Prodmap,  обновляем загрузку Prodmap и её сцепку с bid_df

In [31]:
qty_col = "Оборот в штуках"
pi_df = pivoted(qty_col)

In [32]:
# Сохраняем получивщийся результат в Таблицу с именем дистра
D2 = pi_df.copy()
D2 = D2.rename(columns={qty_col: 'Packs'})

In [33]:
D2.sort_values('Product_name_SS')[:5]

,Distributor,Month,Product_id_SS,Product_name_SS,Packs
32,D2,2021_03,300027136,Allerway 5 mg tab №10,66
34,D2,2021_03,300027158,Allerway 5 mg tab №30,76
49,D2,2021_03,300031318,Allerway Express (1x10s)Tab Russia,2
20,D2,2021_03,300014564,Cetrine tab 10mg №30,617
7,D2,2021_03,300000751,Cetrine tab f/c 10mg №20,1326


##  D3

In [34]:
path = r"C:\Data\Илья\DataScience\D_report\03_D"
col_name ='наименование товара'
qty_col = "количество"

In [35]:
big_df = files_from_folder_to_df(path)
big_df['Month'] = cur_per
big_df['Distributor'] = 'D3'
big_df_ss = delete_totals(col_name)
big_df_ss[col_name] = big_df_ss[col_name].str.lower()
big_df_ss = pd.merge(big_df_ss, prodmap, how ='left', left_on = col_name, right_on = 'Product_name_origin')
find_not_mapped(col_name)

ОК. Все продукты смепплены!


In [36]:
pi_df = pivoted(qty_col)
D3 = pi_df.copy()

In [37]:
D3 = D3.rename(columns={qty_col: 'Packs'})
D3.sort_values('Product_name_SS')[:5]

,Distributor,Month,Product_id_SS,Product_name_SS,Packs
58,D3,2021_03,300027136,Allerway 5 mg tab №10,115
60,D3,2021_03,300027158,Allerway 5 mg tab №30,83
79,D3,2021_03,300031318,Allerway Express (1x10s)Tab Russia,70
15,D3,2021_03,300001611,Bion 3 tab №10,80
17,D3,2021_03,300001761,Bion 3 tab №30,972


## D4

In [38]:
path = r"C:\Data\Илья\DataScience\D_report\04_D"
col_name ='Товар'
qty_col = "Продажи, шт."

In [39]:
big_df = files_from_folder_to_df(path)
big_df['Month'] = cur_per
big_df['Distributor'] = 'D4'
big_df_ss = delete_totals(col_name)
big_df_ss[col_name] = big_df_ss[col_name].str.lower()
big_df_ss = pd.merge(big_df_ss, prodmap, how ='left', left_on = col_name, right_on = 'Product_name_origin')
find_not_mapped(col_name)

ОК. Все продукты смепплены!


In [40]:
pi_df = pivoted(qty_col)
D4 = pi_df.copy()

In [41]:
D4 = D4.rename(columns={qty_col: 'Packs'})
D4.sort_values('Product_name_SS')[:5]

,Distributor,Month,Product_id_SS,Product_name_SS,Packs
37,D4,2021_03,300027136,Allerway 5 mg tab №10,132
39,D4,2021_03,300027158,Allerway 5 mg tab №30,39
55,D4,2021_03,300031318,Allerway Express (1x10s)Tab Russia,42
11,D4,2021_03,300001611,Bion 3 tab №10,16
23,D4,2021_03,300014564,Cetrine tab 10mg №30,1137


## D5

In [42]:
path = r"C:\Data\Илья\DataScience\D_report\05_D"
col_name ='Название'
qty_col = "Упаковок"

In [43]:
big_df = files_from_folder_to_df(path)
big_df['Month'] = cur_per
big_df['Distributor'] = 'D5'
big_df_ss = delete_totals(col_name)
big_df_ss[col_name] = big_df_ss[col_name].str.lower()
big_df_ss = pd.merge(big_df_ss, prodmap, how ='left', left_on = col_name, right_on = 'Product_name_origin')
find_not_mapped(col_name)

ОК. Все продукты смепплены!


In [44]:
pi_df = pivoted(qty_col)
D5 = pi_df.copy()

In [45]:
D5 = D5.rename(columns={qty_col: 'Packs'})
D5.sort_values('Product_name_SS')[:5]

,Distributor,Month,Product_id_SS,Product_name_SS,Packs
7,D5,2021_03,300027136,Allerway 5 mg tab №10,2
8,D5,2021_03,300027158,Allerway 5 mg tab №30,3
9,D5,2021_03,300029853,Dexpan plus baby cream 50g,4
4,D5,2021_03,300005873,Ketorol gel 2% 30g,8
6,D5,2021_03,300023895,Ketorol gel 2% 50g,9


## D6

In [46]:
path = r"C:\Data\Илья\DataScience\D_report\06_D"
col_name ='Номенклатура'
qty_col = "Март 2021 г."

In [47]:
big_df = files_from_folder_to_df(path)
big_df['Month'] = cur_per
big_df['Distributor'] = 'D6'
big_df_ss = delete_totals(col_name)
big_df_ss[col_name] = big_df_ss[col_name].str.lower()
big_df_ss = pd.merge(big_df_ss, prodmap, how ='left', left_on = col_name, right_on = 'Product_name_origin')
find_not_mapped(col_name)

ОК. Все продукты смепплены!


In [48]:
pi_df = pivoted(qty_col)
D6 = pi_df.copy()

In [49]:
D6 = D6.rename(columns={qty_col: 'Packs'})
D6.sort_values('Product_name_SS')[:5]

,Distributor,Month,Product_id_SS,Product_name_SS,Packs
28,D6,2021_03,300027136,Allerway 5 mg tab №10,3
30,D6,2021_03,300027158,Allerway 5 mg tab №30,251
9,D6,2021_03,300001761,Bion 3 tab №30,111
18,D6,2021_03,300014564,Cetrine tab 10mg №30,1042
5,D6,2021_03,300000751,Cetrine tab f/c 10mg №20,1123


## D7

In [50]:
path = r"C:\Data\Илья\DataScience\D_report\07_D"
col_name ='Товар'
qty_col = "Количество"

In [51]:
big_df = files_from_folder_to_df(path)
big_df['Month'] = cur_per
big_df['Distributor'] = 'D7'
big_df_ss = delete_totals(col_name)
big_df_ss[col_name] = big_df_ss[col_name].str.lower()
big_df_ss = pd.merge(big_df_ss, prodmap, how ='left', left_on = col_name, right_on = 'Product_name_origin')
find_not_mapped(col_name)

ОК. Все продукты смепплены!


In [52]:
pi_df = pivoted(qty_col)
D7 = pi_df.copy()

In [53]:
D7 = D7.rename(columns={qty_col: 'Packs'})
D7.sort_values('Product_name_SS')[:5]

,Distributor,Month,Product_id_SS,Product_name_SS,Packs
58,D7,2021_03,300027136,Allerway 5 mg tab №10,96
60,D7,2021_03,300027158,Allerway 5 mg tab №30,94
79,D7,2021_03,300031318,Allerway Express (1x10s)Tab Russia,40
15,D7,2021_03,300001611,Bion 3 tab №10,75
17,D7,2021_03,300001761,Bion 3 tab №30,98


## D8

In [54]:
path = r"C:\Data\Илья\DataScience\D_report\08_D"
col_name ='Product Name'
qty_col = "Количество"

In [55]:
big_df = files_from_folder_to_df(path)
big_df['Month'] = cur_per
big_df['Distributor'] = 'D8'
big_df_ss = delete_totals(col_name)
big_df_ss[col_name] = big_df_ss[col_name].str.lower()
big_df_ss = pd.merge(big_df_ss, prodmap, how ='left', left_on = col_name, right_on = 'Product_name_origin')
find_not_mapped(col_name)

ОК. Все продукты смепплены!


In [56]:
pi_df = pivoted(qty_col)
D8 = pi_df.copy()

In [57]:
D8 = D8.rename(columns={qty_col: 'Packs'})
D8.sort_values('Product_name_SS')[:5]

,Distributor,Month,Product_id_SS,Product_name_SS,Packs
37,D8,2021_03,300027136,Allerway 5 mg tab №10,913
39,D8,2021_03,300027158,Allerway 5 mg tab №30,139
56,D8,2021_03,300031318,Allerway Express (1x10s)Tab Russia,508
11,D8,2021_03,300001761,Bion 3 tab №30,356
23,D8,2021_03,300014564,Cetrine tab 10mg №30,893


## D9

In [58]:
path = r"C:\Data\Илья\DataScience\D_report\09_D"
col_name ='Наименование'
qty_col = "Кол-во"

In [59]:
big_df = files_from_folder_to_df(path)
big_df['Month'] = cur_per
big_df['Distributor'] = 'D9'
big_df_ss = delete_totals(col_name)
big_df_ss[col_name] = big_df_ss[col_name].str.lower()
big_df_ss = pd.merge(big_df_ss, prodmap, how ='left', left_on = col_name, right_on = 'Product_name_origin')
find_not_mapped(col_name)

ОК. Все продукты смепплены!


In [60]:
pi_df = pivoted(qty_col)
D9 = pi_df.copy()

In [61]:
D9 = D9.rename(columns={qty_col: 'Packs'})
D9.sort_values('Product_name_SS')[:5]

,Distributor,Month,Product_id_SS,Product_name_SS,Packs
17,D9,2021_03,300027136,Allerway 5 mg tab №10,120
19,D9,2021_03,300027158,Allerway 5 mg tab №30,6
27,D9,2021_03,300031318,Allerway Express (1x10s)Tab Russia,16
5,D9,2021_03,300001611,Bion 3 tab №10,4
6,D9,2021_03,300001761,Bion 3 tab №30,217


## D10

In [62]:
path = r"C:\Data\Илья\DataScience\D_report\10_D"
col_name ='ТН'
qty_col = "кол-во отгружено"

In [63]:
big_df = files_from_folder_to_df(path)
big_df['Month'] = cur_per
big_df['Distributor'] = 'D10'
big_df_ss = delete_totals(col_name)
big_df_ss[col_name] = big_df_ss[col_name].str.lower()
big_df_ss = pd.merge(big_df_ss, prodmap, how ='left', left_on = col_name, right_on = 'Product_name_origin')
find_not_mapped(col_name)

ОК. Все продукты смепплены!


In [64]:
pi_df = pivoted(qty_col)
D10 = pi_df.copy()

In [65]:
D10 = D10.rename(columns={qty_col: 'Packs'})
D10.sort_values('Product_name_SS')[:5]

,Distributor,Month,Product_id_SS,Product_name_SS,Packs
1,D10,2021_03,300001157,Ciprolet inf 2mg/ml 100ml №1,1460
9,D10,2021_03,300030215,Enam tab 10mg №20,1050
10,D10,2021_03,300030216,"Enam tab 2,5mg №20",1151
11,D10,2021_03,300030235,Enam tab 5mg №20,3015
8,D10,2021_03,300020052,Ibuclin tab f/c №10,69


## D11

In [66]:
path = r"C:\Data\Илья\DataScience\D_report\11_D"
col_name ='Розничная точка'
qty_col = "Итог"

In [67]:
big_df = files_from_folder_to_df(path)
big_df['Month'] = cur_per
big_df['Distributor'] = 'D11'
big_df_ss = delete_totals(col_name)
big_df_ss[col_name] = big_df_ss[col_name].str.lower()
big_df_ss = pd.merge(big_df_ss, prodmap, how ='left', left_on = col_name, right_on = 'Product_name_origin')
find_not_mapped(col_name)

ОК. Все продукты смепплены!


In [68]:
pi_df = pivoted(qty_col)
D11 = pi_df.copy()

In [69]:
D11 = D11.rename(columns={qty_col: 'Packs'})
D11.sort_values('Product_name_SS')[:5]

,Distributor,Month,Product_id_SS,Product_name_SS,Packs
36,D11,2021_03,300027136,Allerway 5 mg tab №10,887
38,D11,2021_03,300027158,Allerway 5 mg tab №30,276
12,D11,2021_03,300001761,Bion 3 tab №30,972
24,D11,2021_03,300014564,Cetrine tab 10mg №30,363
7,D11,2021_03,300000751,Cetrine tab f/c 10mg №20,912


# 6) Сцепляем в единый "monthly_df"

In [70]:
all = ([D1, D2, D3, D4, D5, D6, D7, D8, D9, D10, D11])
monthly_df = pd.concat(all, ignore_index=True)

In [71]:
monthly_df = monthly_df.rename(columns={'Packs': 'S2103P'})

In [72]:
monthly_df.sample(10)

,Distributor,Month,Product_id_SS,Product_name_SS,S2103P
222,D3,2021_03,300032412,Ketorol inj 30mg/ml 1 ml №10,152
294,D5,2021_03,300005873,Ketorol gel 2% 30g,8
579,D11,2021_03,300028322,"Nasivin nasal drops 0,025%",974
162,D3,2021_03,300004174,Mitotax inf 30 mg/5 ml №1,13
578,D11,2021_03,300027714,Nise granules 100 mg №9,713
286,D4,2021_03,300032413,NISE ACTIVGEL 20g Russia,150
235,D4,2021_03,300000604,Ciprolet tab f/c 500 mg №10,991
440,D8,2021_03,300001258,Ciprolet ED 5ml №1,208
153,D3,2021_03,300001157,Ciprolet inf 2mg/ml 100ml №1,128
137,D2,2021_03,300032412,Ketorol inj 30mg/ml 1 ml №10,773


# 7) Предпроверка
что все продукты из Monthly существуют в списке продуктов main

In [73]:
monthly_sku = monthly_df['Product_id_SS'].unique()
main_sku = main['Product_id_SS'].unique()

not_found = []
for x in monthly_sku:
    if x not in main_sku:
        not_found.append(x)
not_found

[300000108, 300007397]

# 8) Присоединяем montly_df к main

In [74]:
main = pd.merge(main, monthly_df[['Distributor', 'Product_id_SS','S2103P']], how = 'left',
                          left_on = ['Distributor', 'Product_id_SS'], right_on = ['Distributor', 'Product_id_SS'])

In [75]:
main.S2103P = main.S2103P.fillna(0)

In [76]:
main.head(2)

,Distributor,Product_id_SS,Product_name_SS,S1904P,S1905P,S1906P,S1907P,S1908P,S1909P,S1910P,...,S2006P,S2007P,S2008P,S2009P,S2010P,S2011P,S2012P,S2101P,S2102P,S2103P
0,D1,300027136,Allerway 5 mg tab №10,319,986,443,800,227,92,77,...,691,541,722,89,325,985,50,15,634,150.00
1,D1,300027158,Allerway 5 mg tab №30,571,321,527,224,893,878,412,...,245,480,648,861,277,330,434,465,336,158.00


# 9) Задаём Cur_mon, Cur_mon_LY   !!!!!!!!!!!!!

In [77]:
mon = 'Mar'  #  для YTD
main['Cur_mon_LY'] = main.S2003P
main['Cur_mon'] = main.S2103P
monthly_df['Cur_mon'] = monthly_df.S2103P  #  для Проверки!

In [78]:
main.head(2)

,Distributor,Product_id_SS,Product_name_SS,S1904P,S1905P,S1906P,S1907P,S1908P,S1909P,S1910P,...,S2008P,S2009P,S2010P,S2011P,S2012P,S2101P,S2102P,S2103P,Cur_mon_LY,Cur_mon
0,D1,300027136,Allerway 5 mg tab №10,319,986,443,800,227,92,77,...,722,89,325,985,50,15,634,150.00,453,150.00
1,D1,300027158,Allerway 5 mg tab №30,571,321,527,224,893,878,412,...,648,861,277,330,434,465,336,158.00,187,158.00


In [79]:
monthly_df.head(2)

,Distributor,Month,Product_id_SS,Product_name_SS,S2103P,Cur_mon
0,D1,2021_03,300000108,Irnocam 100 mg,15,15
1,D1,2021_03,300000169,Ibuclin junior tab №20,287,287


# 10) Постпроверка
- A) по сумме,
- B) дистрам,
- C) продуктам

### А) по общей сумме

In [80]:
monthly_sum = monthly_df.Cur_mon.sum()
main_sum = main.Cur_mon.sum()
main_sum==monthly_sum

False

In [81]:
main_sum

304713.0

### B) по дистрам

In [82]:
mon_dist = monthly_df.groupby(['Distributor']).sum().reset_index()[['Distributor','Cur_mon']]
main_dist = main.groupby(['Distributor']).sum().reset_index()[['Distributor', 'Cur_mon']]
check_dist = pd.merge(mon_dist, main_dist, how ='outer', on = 'Distributor')
check_dist['Diff'] = check_dist.Cur_mon_x - check_dist.Cur_mon_y
check_dist

,Distributor,Cur_mon_x,Cur_mon_y,Diff
0,D1,49954,"49,930.00",24.00
1,D10,14450,"14,450.00",0.00
2,D11,34492,"34,492.00",0.00
3,D2,22014,"22,014.00",0.00
4,D3,9721,"9,701.00",20.00
5,D4,22741,"22,741.00",0.00
6,D5,176,176.00,0.00
7,D6,87950,"87,950.00",0.00
8,D7,6876,"6,862.00",14.00
9,D8,35491,"35,491.00",0.00


### C) по продуктам

In [83]:
monthly_prods = monthly_df.groupby(['Product_name_SS',
                                    'Product_id_SS']).sum().reset_index()[['Product_name_SS','Product_id_SS','Cur_mon']]
main_prods = main.groupby(['Product_id_SS',
    'Product_name_SS']).sum().reset_index()[['Product_name_SS','Product_id_SS', 'Cur_mon']]

check_prods = pd.merge(monthly_prods, main_prods, how ='outer', on=['Product_name_SS', 'Product_id_SS'])
check_prods['Diff'] = check_prods.Cur_mon_x - check_prods.Cur_mon_y

In [84]:
check_prods.head(2)

,Product_name_SS,Product_id_SS,Cur_mon_x,Cur_mon_y,Diff
0,Allerway 5 mg tab №10,300027136,"2,484.00","2,484.00",0.00
1,Allerway 5 mg tab №30,300027158,"1,125.00","1,125.00",0.00


#### Выгружаем main в xlsx
для использования в следующий месяц

In [85]:
# main.to_excel(r'C:\Data\Илья\DataScience\D_report\Data_x.xlsx', index = False)

# 11) Расчёт Sales YTD & YTD_LY

In [87]:
main['YTD_LY'] = YTD_1920(mon)
main['YTD'] = YTD_2021(mon)

# 12) Оставляем Cur_mon и YTD

In [88]:
selected_packs = main[['Distributor', 'Product_name_SS', 'Product_id_SS', 'Cur_mon_LY', 'Cur_mon', 'YTD_LY', 'YTD']].copy()
selected_packs['Measure'] = 'Packs'

In [89]:
selected_packs.head(2)

,Distributor,Product_name_SS,Product_id_SS,Cur_mon_LY,Cur_mon,YTD_LY,YTD,Measure
0,D1,Allerway 5 mg tab №10,300027136,453,150.00,4716,"5,776.00",Packs
1,D1,Allerway 5 mg tab №30,300027158,187,158.00,6411,"5,564.00",Packs


# 13) Рассчитываем Тотал по дистрам

In [90]:
Total = selected_packs.groupby(['Product_id_SS', 'Product_name_SS', 'Measure']).sum().reset_index()
Total['Distributor'] = 'Total'

In [91]:
Total.head(2)

,Product_id_SS,Product_name_SS,Measure,Cur_mon_LY,Cur_mon,YTD_LY,YTD,Distributor
0,300000169,Ibuclin junior tab №20,Packs,4427,"4,043.00",66328,"66,948.00",Total
1,300000269,Novigan tab f/c №20,Packs,5788,"2,410.00",66393,"66,386.00",Total


# 14) Присоединяем Тотал к _ selected_packs

In [92]:
selected_packs = pd.concat([selected_packs, Total])

In [93]:
selected_packs.sample(5)

,Distributor,Product_name_SS,Product_id_SS,Cur_mon_LY,Cur_mon,YTD_LY,YTD,Measure
195,D11,Femibion Natal Care II tab №30 + caps №30,300008741,458,0.00,6435,"5,298.00",Packs
637,D6,Levolet R tab f/c 500mg №10,300004774,468,1.00,5015,"4,446.00",Packs
280,D2,Femibion Natal Care I tab f/c №30,300013022,519,166.00,6264,"5,950.00",Packs
676,D6,Omez lyoph. inf 40mg №1,300008393,147,179.00,7899,"3,840.00",Packs
477,D4,Nasivin Metered Spray 0.005%10ml,300029322,418,243.00,7007,"5,606.00",Packs


# 15) Расчёт Руб
Умножаем на цены

In [94]:
selected_rub = pd.merge(selected_packs, prices[['Product_id_SS','Price_fix_RUB']], how = 'left', on = 'Product_id_SS')

In [95]:
selected_rub['Cur_mon_LY'] = selected_rub.Cur_mon_LY.multiply(selected_rub.Price_fix_RUB)
selected_rub['Cur_mon'] = selected_rub.Cur_mon.multiply(selected_rub.Price_fix_RUB)
selected_rub['YTD_LY'] = selected_rub.YTD_LY.multiply(selected_rub.Price_fix_RUB)
selected_rub['YTD'] = selected_rub.YTD.multiply(selected_rub.Price_fix_RUB)
selected_rub['Measure'] = 'RUB'
selected_rub = selected_rub.drop('Price_fix_RUB', axis = 1)

In [96]:
selected_rub.sample(2)

,Distributor,Product_name_SS,Product_id_SS,Cur_mon_LY,Cur_mon,YTD_LY,YTD,Measure
660,D6,Nise gel 1% 100g,300021569,527176,"6,734.00",6075992,"5,020,678.00",RUB
132,D10,Momat Rino Advance 75,300034607,133056,0.00,1855623,"1,510,278.00",RUB


### Собираем Packs & RUB в единый df

In [97]:
selected_all = pd.concat([selected_packs, selected_rub]).reset_index()
# важно обновить Индекс! иначе он дублируется

In [98]:
selected_all= selected_all.drop('index', axis = 1)

In [99]:
selected_all.sample(2)

,Distributor,Product_name_SS,Product_id_SS,Cur_mon_LY,Cur_mon,YTD_LY,YTD,Measure
1778,D7,Nise gel 1% 100g,300021569,47138,"82,732.00",7381426,"6,088,498.00",RUB
1097,D1,Normase sirup 200ml,300001766,793305,"911,340.00",3727710,"6,384,870.00",RUB


# 16) Добавляем Тотал по продуктам в Уп и Руб 

In [100]:
total_all = selected_all[['Distributor', 'Cur_mon_LY', 'Cur_mon', 'YTD_LY', 'YTD', 'Measure']].groupby(['Measure','Distributor']).sum().reset_index()

In [101]:
total_all['Product_name_SS'] = 'Total'
total_all['Product_id_SS'] = 'Total'

In [102]:
total_all.sample(2)

,Measure,Distributor,Cur_mon_LY,Cur_mon,YTD_LY,YTD,Product_name_SS,Product_id_SS
10,Packs,D9,41317,"20,906.00",510591,"484,038.00",Total,Total
16,RUB,D3,18317539,"4,991,803.00",254837526,"241,878,807.00",Total,Total


In [103]:
selected_all = pd.concat([selected_all, total_all]).reset_index().drop('index', axis=1)

In [104]:
selected_all.sample(2)

,Distributor,Product_name_SS,Product_id_SS,Cur_mon_LY,Cur_mon,YTD_LY,YTD,Measure
550,D5,Levolet R tab f/c 250mg №10,300004773,970,11.00,5887,"5,426.00",Packs
474,D4,Momat Rino 60,300034653,924,0.00,4725,"5,321.00",Packs


# 16) Добавляем приросты

In [105]:
selected_all['Gr_mon'] = [Grt_month(row) for row in range(len(selected_all))]
selected_all['Gr_ytd'] = [Grt_YTD(row) for row in range(len(selected_all))]

In [106]:
selected_all.sample(2)

,Distributor,Product_name_SS,Product_id_SS,Cur_mon_LY,Cur_mon,YTD_LY,YTD,Measure,Gr_mon,Gr_ytd
1440,D3,Niselat tab 600mg №20,300008097,214830,"63,240.00",6148230,"3,073,650.00",RUB,-0.71,-0.50
292,D2,Levolet R tab f/c 250mg №10,300004773,872,0.00,7207,"6,845.00",Packs,-1.00,-0.05


# 17) Выстраиваем в нужный вид
Перевод из Плоского списка в Отчётный вид

In [107]:
D = selected_all.pivot_table(index = ['Product_name_SS','Product_id_SS'], columns = ['Distributor', 'Measure'], 
                    values = ['Cur_mon', 'Cur_mon_LY','Gr_mon', 'YTD', 'YTD_LY', 'Gr_ytd'], aggfunc = 'mean', fill_value=0)

In [108]:
D.sample(2)

Cur_mon                          \
Distributor                                    D1           D10       D11   
Measure                                     Packs     RUB Packs RUB Packs   
Product_name_SS             Product_id_SS                                   
Momat Rino Advance 75       300034607         253   58443     0   0     0   
Levolet R tab f/c 750mg №10 300022893         495  222750     0   0   360   

                                                                            \
Distributor                                          D2          D3          
Measure                                       RUB Packs   RUB Packs    RUB   
Product_name_SS             Product_id_SS                                    
Momat Rino Advance 75       300034607           0     0     0    29   6699   
Levolet R tab f/c 750mg №10 300022893      162000    19  8550    65  29250   

                                           ... YTD_LY                          \
Distributor                                ...     D6             D7            
Measure                                    ...  Packs      RUB Packs      RUB   
Product_name_SS             Product_id_SS  ...                                  
Momat Rino Advance 75       300034607      ...   7503  1733193  4260   984060   
Levolet R tab f/c 750mg №10 300022893      ...   6815  3066750  6761  3042450   

                                                                         \
Distributor                                  D8             D9            
Measure                                   Packs      RUB Packs      RUB   
Product_name_SS             Product_id_SS                                 
Momat Rino Advance 75       300034607      5593  1291983  5828  1346268   
Levolet R tab f/c 750mg №10 300022893      6333  2849850  3849  1732050   

                                                            
Distributor                                Total            
Measure                                    Packs       RUB  
Product_name_SS             Product_id_SS                   
Momat Rino Advance 75       300034607      65707  15178317  
Levolet R tab f/c 750mg №10 300022893      65685  29558250  

[2 rows x 144 columns]

In [109]:
D = D.reset_index().copy()

In [110]:
D.columns = ['_'.join(col) for col in D.columns]

In [111]:
D = D.rename(columns ={'Product_name_SS__': 'Product_name_SS', 'Product_id_SS__':'Product_id_SS'})

In [112]:
D.head(2)

,Product_name_SS,Product_id_SS,Cur_mon_D1_Packs,Cur_mon_D1_RUB,Cur_mon_D10_Packs,Cur_mon_D10_RUB,Cur_mon_D11_Packs,Cur_mon_D11_RUB,Cur_mon_D2_Packs,Cur_mon_D2_RUB,...,YTD_LY_D6_Packs,YTD_LY_D6_RUB,YTD_LY_D7_Packs,YTD_LY_D7_RUB,YTD_LY_D8_Packs,YTD_LY_D8_RUB,YTD_LY_D9_Packs,YTD_LY_D9_RUB,YTD_LY_Total_Packs,YTD_LY_Total_RUB
0,Allerway 5 mg tab №10,300027136,150,29850,0,0,887,176513,66,13134,...,6051,1204149,6431,1279769,4430,881570,4576,910624,63464,12629336
1,Allerway 5 mg tab №30,300027158,158,127980,0,0,276,223560,76,61560,...,6301,5103810,3379,2736990,6338,5133780,5804,4701240,65803,53300430


# Проверка итогов

In [113]:
check_total = D[['Product_name_SS', 'Product_id_SS',
    'Cur_mon_Total_Packs',  'Cur_mon_Total_RUB', 'Cur_mon_LY_Total_Packs', 'Cur_mon_LY_Total_RUB',
    'YTD_Total_Packs', 'YTD_Total_RUB', 'YTD_LY_Total_Packs', 'YTD_LY_Total_RUB']]

In [114]:
check_total[check_total.Product_name_SS=='Total']

,Product_name_SS,Product_id_SS,Cur_mon_Total_Packs,Cur_mon_Total_RUB,Cur_mon_LY_Total_Packs,Cur_mon_LY_Total_RUB,YTD_Total_Packs,YTD_Total_RUB,YTD_LY_Total_Packs,YTD_LY_Total_RUB
86,Total,Total,304713,163891851,458216,231366053,5502708,2766813293,5651237,2850686690


In [115]:
check_total.head()

,Product_name_SS,Product_id_SS,Cur_mon_Total_Packs,Cur_mon_Total_RUB,Cur_mon_LY_Total_Packs,Cur_mon_LY_Total_RUB,YTD_Total_Packs,YTD_Total_RUB,YTD_LY_Total_Packs,YTD_LY_Total_RUB
0,Allerway 5 mg tab №10,300027136,2484,494316,4581,911619,63595,12655405,63464,12629336
1,Allerway 5 mg tab №30,300027158,1125,911250,4497,3642570,59849,48477690,65803,53300430
2,Allerway Express (1x10s)Tab Russia,300031318,744,303552,6134,2502672,58277,23777016,62394,25456752
3,Bion 3 tab №10,300001611,259,95312,6482,2385376,62471,22989328,67822,24958496
4,Bion 3 tab №30,300001761,2826,1251918,7090,3140870,59998,26579114,65405,28974415


# 18) Делим D на:  month  и  YTD 

In [116]:
month = D[['Product_name_SS', 'Product_id_SS',
    'Cur_mon_Total_Packs', 'Cur_mon_Total_RUB', 'Gr_mon_Total_RUB',    
    'Cur_mon_D1_Packs',  'Cur_mon_D1_RUB',   'Gr_mon_D1_RUB', 
    'Cur_mon_D2_Packs',  'Cur_mon_D2_RUB',   'Gr_mon_D2_RUB',
    'Cur_mon_D3_Packs',  'Cur_mon_D3_RUB',   'Gr_mon_D3_RUB',   
    'Cur_mon_D4_Packs',  'Cur_mon_D4_RUB',   'Gr_mon_D4_RUB',  
    'Cur_mon_D5_Packs',  'Cur_mon_D5_RUB',   'Gr_mon_D5_RUB',
    'Cur_mon_D6_Packs',  'Cur_mon_D6_RUB',   'Gr_mon_D6_RUB',           
    'Cur_mon_D7_Packs',  'Cur_mon_D7_RUB',   'Gr_mon_D7_RUB',
    'Cur_mon_D8_Packs',  'Cur_mon_D8_RUB',   'Gr_mon_D8_RUB',
    'Cur_mon_D9_Packs',  'Cur_mon_D9_RUB',   'Gr_mon_D9_RUB', 
    'Cur_mon_D10_Packs', 'Cur_mon_D10_RUB',  'Gr_mon_D10_RUB',
    'Cur_mon_D11_Packs', 'Cur_mon_D11_RUB',  'Gr_mon_D11_RUB']].copy()

In [117]:
YTD = D[['Product_name_SS', 'Product_id_SS',
    'YTD_Total_Packs', 'YTD_Total_RUB', 'Gr_ytd_Total_RUB',    
    'YTD_D1_Packs',  'YTD_D1_RUB',   'Gr_ytd_D1_RUB', 
    'YTD_D2_Packs',  'YTD_D2_RUB',   'Gr_ytd_D2_RUB',
    'YTD_D3_Packs',  'YTD_D3_RUB',   'Gr_ytd_D3_RUB',   
    'YTD_D4_Packs',  'YTD_D4_RUB',   'Gr_ytd_D4_RUB',  
    'YTD_D5_Packs',  'YTD_D5_RUB',   'Gr_ytd_D5_RUB',
    'YTD_D6_Packs',  'YTD_D6_RUB',   'Gr_ytd_D6_RUB',           
    'YTD_D7_Packs',  'YTD_D7_RUB',   'Gr_ytd_D7_RUB',
    'YTD_D8_Packs',  'YTD_D8_RUB',   'Gr_ytd_D8_RUB',
    'YTD_D9_Packs',  'YTD_D9_RUB',   'Gr_ytd_D9_RUB', 
    'YTD_D10_Packs', 'YTD_D10_RUB',  'Gr_ytd_D10_RUB',
    'YTD_D11_Packs', 'YTD_D11_RUB',  'Gr_ytd_D11_RUB']].copy()

In [118]:
month.head()

,Product_name_SS,Product_id_SS,Cur_mon_Total_Packs,Cur_mon_Total_RUB,Gr_mon_Total_RUB,Cur_mon_D1_Packs,Cur_mon_D1_RUB,Gr_mon_D1_RUB,Cur_mon_D2_Packs,Cur_mon_D2_RUB,...,Gr_mon_D8_RUB,Cur_mon_D9_Packs,Cur_mon_D9_RUB,Gr_mon_D9_RUB,Cur_mon_D10_Packs,Cur_mon_D10_RUB,Gr_mon_D10_RUB,Cur_mon_D11_Packs,Cur_mon_D11_RUB,Gr_mon_D11_RUB
0,Allerway 5 mg tab №10,300027136,2484,494316,-0.46,150,29850,-0.67,66,13134,...,8.82,120,23880,9.91,0,0,-1.00,887,176513,14.03
1,Allerway 5 mg tab №30,300027158,1125,911250,-0.75,158,127980,-0.16,76,61560,...,-0.78,6,4860,-0.96,0,0,-1.00,276,223560,-0.24
2,Allerway Express (1x10s)Tab Russia,300031318,744,303552,-0.88,66,26928,-0.91,2,816,...,-0.07,16,6528,-0.95,0,0,-1.00,0,0,-1.00
3,Bion 3 tab №10,300001611,259,95312,-0.96,84,30912,-0.90,0,0,...,-1.00,4,1472,-0.99,0,0,-1.00,0,0,-1.00
4,Bion 3 tab №30,300001761,2826,1251918,-0.60,100,44300,-0.76,0,0,...,-0.32,217,96131,0.02,0,0,-1.00,972,430596,0.13


In [119]:
YTD.head()

,Product_name_SS,Product_id_SS,YTD_Total_Packs,YTD_Total_RUB,Gr_ytd_Total_RUB,YTD_D1_Packs,YTD_D1_RUB,Gr_ytd_D1_RUB,YTD_D2_Packs,YTD_D2_RUB,...,Gr_ytd_D8_RUB,YTD_D9_Packs,YTD_D9_RUB,Gr_ytd_D9_RUB,YTD_D10_Packs,YTD_D10_RUB,Gr_ytd_D10_RUB,YTD_D11_Packs,YTD_D11_RUB,Gr_ytd_D11_RUB
0,Allerway 5 mg tab №10,300027136,63595,12655405,0.00,5776,1149424,0.22,4753,945847,...,0.81,4903,975697,0.07,5728,1139872,0.15,6373,1268227,-0.20
1,Allerway 5 mg tab №30,300027158,59849,48477690,-0.09,5564,4506840,-0.13,5093,4125330,...,-0.30,5489,4446090,-0.05,6440,5216400,-0.03,5697,4614570,-0.06
2,Allerway Express (1x10s)Tab Russia,300031318,58277,23777016,-0.07,4708,1920864,-0.23,7290,2974320,...,0.02,3800,1550400,-0.41,4829,1970232,-0.15,4194,1711152,-0.18
3,Bion 3 tab №10,300001611,62471,22989328,-0.08,6627,2438736,-0.05,4847,1783696,...,-0.21,4794,1764192,0.00,5975,2198800,-0.13,7432,2734976,0.54
4,Bion 3 tab №30,300001761,59998,26579114,-0.08,4254,1884522,-0.35,6030,2671290,...,-0.12,6130,2715590,0.33,4355,1929265,-0.21,6222,2756346,0.34


In [121]:
#month.to_excel('D_month.xlsx', index = False)
#YTD.to_excel('D_YTD.xlsx', index = False)